In [85]:
import pdftotext
import logging
import spacy
import joblib
import logging
import numpy as np

In [86]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [87]:
class_mapping = {
    'ACCOUNTANT': 0,
    'ADVOCATE': 1,
    'AGRICULTURE': 2,
    'APPAREL': 3,
    'ARTS': 4,
    'AUTOMOBILE': 5,
    'AVIATION': 6,
    'BANKING': 7,
    'BPO': 8,
    'BUSINESS-DEVELOPMENT': 9,
    'CHEF': 10,
    'CONSTRUCTION': 11,
    'CONSULTANT': 12,
    'DESIGNER': 13,
    'DIGITAL-MEDIA': 14,
    'ENGINEERING': 15,
    'FINANCE': 16,
    'FITNESS': 17,
    'HEALTHCARE': 18,
    'HR': 19,
    'INFORMATION-TECHNOLOGY': 20,
    'PUBLIC-RELATIONS': 21,
    'SALES': 22,
    'TEACHER': 23
}

In [88]:
model = joblib.load("ai/model.pkl")
vectorizer = joblib.load("ai/vector.pkl")
nlp = spacy.load("en_core_web_lg")

In [89]:
def remove_stopwords_and_lemmatize(text):
    cleaned_text = ' '.join(text.strip().split())
    doc = nlp(cleaned_text.lower())
    tokens_lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens_lemmatized)

In [90]:
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, "rb") as f:
        pdf = pdftotext.PDF(f)
    return "\n\n".join(pdf)

In [91]:
def process_pdf_and_classify(pdf_file):
    text = extract_text_from_pdf(pdf_file)
    preprocessed_text = remove_stopwords_and_lemmatize(text)
    X = np.array([preprocessed_text])
    X_vec = vectorizer.transform(X)
    probabilities = model.predict_proba(X_vec)
    return probabilities

In [92]:
def top_probabilities(probabilities):
    top_indices = np.argsort(probabilities[0])[::-1][:5]
    print("Top 5 probabilidades:")
    for idx in top_indices:
        class_name = [key for key, value in class_mapping.items() if value == idx][0]
        prob_percent = probabilities[0][idx] * 100
        print(f"{class_name}: {prob_percent:.2f}%")

In [96]:
pdf_file = 'teste_ia/public-relations.pdf'
probabilities = process_pdf_and_classify(pdf_file)

In [97]:
top_probabilities(probabilities)

Top 5 probabilidades:
PUBLIC-RELATIONS: 99.30%
HR: 0.29%
ACCOUNTANT: 0.20%
HEALTHCARE: 0.07%
ADVOCATE: 0.03%
